# Scavenger

## Setup

The setup script `scavenger-setup.sh` starts the `minikube` server, creates a namespace `scavenger`, sets the current context to the newly created namespace, and creates the resources defined in the `k8s` directory.

In [87]:
%pycat scavenger-setup.sh

# Run only once minikube and relevant dependencies installed
if [ ! -f /usr/local/bin/minikube ]; then
  echo "Minikube not found. Please follow installation instructions in README.md"
fi
minikube start
kubectl create namespace scavenger
kubectl config set-context --current --namespace=scavenger
cd k8s
IFS=$'\n\t'
for f in $(ls)
do
        until kubectl create -f $f 2> /dev/null
        do
                sleep 10
        done
done


Let us execute the setup script

In [46]:
%%bash
./scavenger-setup.sh

* minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
* Using the docker driver based on existing profile
* Starting control plane node minikube in cluster minikube
* Pulling base image ...
* Restarting existing docker container for "minikube" ...
* Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
* Configuring bridge CNI (Container Networking Interface) ...
* Verifying Kubernetes components...
  - Using image gcr.io/k8s-minikube/storage-provisioner:v5
  - Using image registry.k8s.io/metrics-server/metrics-server:v0.6.4
  - Using image docker.io/kubernetesui/dashboard:v2.7.0
  - Using image docker.io/kubernetesui/metrics-scraper:v1.0.8
* Some dashboard features require the metrics-server addon. To enable all features please run:

	minikube addons enable metrics-server	


* Enabled addons: storage-provisioner, default-storageclass, metrics-server, dashboard
* Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
namespace/scavenger created
Context "mi

## Explore

### Mark I: New Beginnings

__1. What are the total number of Pods?__

In [78]:
!kubectl get pods

NAME                               READY   STATUS             RESTARTS        AGE
nginx                              1/1     Running            0               10m
nginx-deployment-55f598f8d-bjrqh   1/1     Running            0               10m
nginx-deployment-55f598f8d-km79w   1/1     Running            0               10m
nginx-deployment-55f598f8d-wtcnh   1/1     Running            0               10m
postgres                           0/1     CrashLoopBackOff   6 (4m34s ago)   10m


In [48]:
%%bash
NB_PODS_TOTAL=$(kubectl get pods | tail -n +2 | wc -l)
echo "Number of Total Pods: $NB_PODS_TOTAL"

Number of Total Pods: 5


__2. How many namespaces are there?__

In [49]:
!kubectl get namespaces

NAME                   STATUS   AGE
default                Active   2d
kube-node-lease        Active   53m
kube-public            Active   2d
kube-system            Active   2d
kubernetes-dashboard   Active   53m
scavenger              Active   13s


In [50]:
%%bash
NB_NAMESPACES=$(kubectl get namespaces | tail -n +2 | wc -l)
echo "Number of Namespaces: $NB_NAMESPACES"

Number of Namespaces: 6


__3. How many nodes do we have?__

In [51]:
!kubectl get nodes

NAME       STATUS   ROLES           AGE   VERSION
minikube   Ready    control-plane   2d    v1.27.4


In [52]:
%%bash
NB_NODES=$(kubectl get nodes | tail -n +2 | wc -l)
echo "Number of Nodes: $NB_NODES"

Number of Nodes: 1


__4. How many failing Pods are there?__

In [69]:
!kubectl get pods | grep -Pv '\s+([1-9]+[\d]*)\/\1\s+'

NAME                               READY   STATUS             RESTARTS      AGE
nginx                              0/1     ImagePullBackOff   0             4m40s
postgres                           0/1     CrashLoopBackOff   5 (86s ago)   4m40s


In [70]:
%%bash
NB_PODS_FAILING=$(kubectl get pods \
  | grep -Pv '\s+([1-9]+[\d]*)\/\1\s+' \
  | tail -n +2 \
  | wc -l
)
echo "Number of Pods Failing: $NB_PODS_FAILING"

Number of Pods Failing: 2


__5. How many Deployments do we have?__

In [71]:
%%bash
kubectl get deployments

NAME               READY   UP-TO-DATE   AVAILABLE   AGE
nginx-deployment   3/3     3            3           5m23s


In [72]:
%%bash
NB_DEPLOYMENTS=$(kubectl get deployments | tail -n +2 | wc -l)
echo $NB_DEPLOYMENTS

1


__6. For the Deployments you discovered, how many ReplicaSets are there in total?__


In [73]:
!kubectl get rs

NAME                         DESIRED   CURRENT   READY   AGE
nginx-deployment-55f598f8d   3         3         3       5m34s


### Mark II: Not-So-New Beginnings

__7. Locate a Postgres Pod within our cluster. Why is it failing? Please provide the variable name(s) that is missing.__

The `postgres` pod was failing because the environment variable `POSTGRESS_PASSWORD` was not set.

In [75]:
!kubectl logs postgres

Error: Database is uninitialized and superuser password is not specified.
       You must specify POSTGRES_PASSWORD to a non-empty value for the
       superuser. For example, "-e POSTGRES_PASSWORD=password" on "docker run".

       You may also use "POSTGRES_HOST_AUTH_METHOD=trust" to allow all
       connections without a password. This is *not* recommended.

       See PostgreSQL documentation about "trust":
       https://www.postgresql.org/docs/current/auth-trust.html


__8. Locate a failed Nginx Pod within our cluster. Why is it failing? Please copy and paste the specific kubectl log you find. Please escape " with \".__

In [74]:
!kubectl logs nginx

Error from server (BadRequest): container "nginx" in pod "nginx" is waiting to start: trying and failing to pull image


### Mark III: Fixing Our Bugs



__9. For the Postgres Pod, can you fix the ./k8s/postgres-pod.yml file to spin up the image correctly? Please add the environment variable needed, with the value "supersecret". added to your lines of code.__

We can fix the `postgres` pod by adding an environment variable for `POSTGRES_PASSWORD`

In [83]:
%%writefile k8s/postgres-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: postgres
spec:
  containers:
    - name: postgres
      image: postgres
      env:
        - name: POSTGRES_PASSWORD
          value: supersecret

Overwriting k8s/postgres-pod.yml


We can delete the failing pod and apply the corrected config to create a new pod.

In [84]:
%%bash
kubectl delete pod postgres
kubectl apply -f k8s/postgres-pod.yml

pod "postgres" deleted
pod/postgres created


We can confirm that the `postgres` pod is running now!

In [85]:
!kubectl get pods

NAME                               READY   STATUS    RESTARTS   AGE
nginx                              1/1     Running   0          4m26s
nginx-deployment-55f598f8d-bjrqh   1/1     Running   0          15m
nginx-deployment-55f598f8d-km79w   1/1     Running   0          15m
nginx-deployment-55f598f8d-wtcnh   1/1     Running   0          15m
postgres                           1/1     Running   0          12s


__10. For the failed Nginx Pod, can you fix the ./k8s/nignx-pod.yml file to spin up the image correctly? Add the lines of code you added/corrected, and please escape " with \".__

The `nginx` pod is failing because of a typo in the image name (`nginz` instead of `nginx`). We can overwrite the config with the correct image name. 

In [76]:
%%writefile k8s/nginx-pod.yml
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  containers:
    - name: nginx
      image: nginx

Overwriting k8s/nginx-pod.yml


We can delete the pod and apply the correct pod configuration.

In [80]:
%%bash
kubectl delete pod nginx
kubectl apply -f k8s/nginx-pod.yml

pod "nginx" deleted
pod/nginx created


We can confirm that the `nginx` pod is up and running after we fixed the pod config.

In [82]:
!kubectl get pods

NAME                               READY   STATUS             RESTARTS     AGE
nginx                              1/1     Running            0            7s
nginx-deployment-55f598f8d-bjrqh   1/1     Running            0            11m
nginx-deployment-55f598f8d-km79w   1/1     Running            0            11m
nginx-deployment-55f598f8d-wtcnh   1/1     Running            0            11m
postgres                           0/1     CrashLoopBackOff   7 (5s ago)   11m


## Teardown

Let us teardown the resources we created by deleting all pods, deployments and namespaces, and stopping the minikube server.

In [45]:
%%bash
kubectl delete pods --all
kubectl delete deployments --all
kubectl delete namespace scavenger
minikube stop

pod "nginx" deleted
pod "nginx-deployment-55f598f8d-g7ntj" deleted
pod "nginx-deployment-55f598f8d-q7p2w" deleted
pod "nginx-deployment-55f598f8d-swb2r" deleted
pod "postgres" deleted
deployment.apps "nginx-deployment" deleted
namespace "scavenger" deleted
* Stopping node "minikube"  ...
* Powering off "minikube" via SSH ...
* 1 node stopped.
